<a href="https://colab.research.google.com/github/adityabharadwaj198/Simple-Quiz-App-Android/blob/master/cudaOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda-repo-ubuntu1604-10-1-local-10.1.105-418.39_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-10-1-local-10.1.105-418.39_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda

--2019-04-02 07:07:19--  https://developer.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda-repo-ubuntu1604-10-1-local-10.1.105-418.39_1.0-1_amd64.deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/10.1/secure/Prod/local_installers/cuda-repo-ubuntu1604-10-1-local-10.1.105-418.39_1.0-1_amd64.deb?yGjgMA5tJ49tpJm8IJsBH2lErVzqjjEyw-DVDD8S9DLtIzcWPy0qGiZS0nwMCtnMJ0m3rZteWT9izdW3R9O9GkE4L4yPiV-RfffXOBCz6rIzOgOf1p-FiPOqevh6oIlgrfMhqf2oHyZDcDTD179uv_7a66P5yZ8qPguYgm27mZy1ppo2LrdrLPEckwjWaw_PEXOCkSCeZvq28lG9HWEk6ECytgMwUwj1J1ICZJM [following]
--2019-04-02 07:07:20--  https://developer.download.nvidia.com/compute/cuda/10.1/secure/Prod/local_installers/cuda-repo-ubuntu1604-10-1-local-10.1.105-418.39_1.0-1_amd64.deb?yGjgMA5tJ49tpJm8IJsBH2lErVzqjjEyw-DVDD8S9DLtIzcWP

In [2]:
!pip install mxnet-cu92

    100% |████████████████████████████████| 448.1MB 50kB/s 
    100% |████████████████████████████████| 61kB 21.6MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 1.0.0 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4


In [1]:
!/usr/local/cuda/bin/nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:17_PST_2019
Cuda compilation tools, release 10.1, V10.1.105


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hmqrzhd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-43z7zzz2/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu
#include <stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void matmul (int *a, int *b, int *c, int wa, int wb)
{
    int blockid = blockIdx.y*gridDim.x + blockIdx.x;
    int threadid = blockid*blockDim.x*blockDim.y + threadIdx.y*blockDim.x + threadIdx.x;
    c[threadid] = a[threadid] + b[threadid];
}
int main ()
{
    int a[16], b[16], c[16];
    for (int i=0; i<16; i++)
      {
          a[i] = i;
          b[i] = i;
          c[i] = 0;
      }
    int wa = 4, wb = 4;
    int *d_a, *d_b, *d_c;
    cudaMalloc((void **)&d_a, 16*sizeof(int));
    cudaMalloc((void **)&d_b, 16*sizeof(int));
    cudaMalloc((void **)&d_c, 16*sizeof(int));
    
    cudaMemcpy(d_a, a, 16*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, 16*sizeof(int), cudaMemcpyHostToDevice);
    dim3 nGridBlock(2,2,1);
    dim3 nBlockThread(2,2,1);
    matmul<<<nGridBlock,nBlockThread>>>(d_a,d_b,d_c,wa,wb);
    cudaMemcpy(c, d_c, 16*sizeof(int), cudaMemcpyDeviceToHost);
    
    for (int i=0; i<16; i++)
    {
        printf("%d   ", c[i]);
    }
    
    cudaFree(d_a);
        cudaFree(d_b);
    cudaFree(d_a);

return 0;
}

'0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   '